# 03. Summary Memory (대화 요약 저장)

이 노트북에서는 **Summary Memory** 방식을 LangChain과 LangGraph로 구현합니다.

**Summary Memory란?**
- 전체 대화를 요약하여 저장
- LLM을 사용하여 대화 내용을 압축
- 적은 토큰으로 전체 맥락 유지

**학습 목표:**
- LLM 기반 대화 요약
- 요약 업데이트 전략
- LangGraph에서 요약 노드 구현

---

## 1. 패키지 설치

In [ ]:
!pip install langchain langchain-anthropic langgraph -q
print("설치 완료!")

설치 완료!


---

## 2. 모델 선택

Claude 4.5 모델 중 하나를 선택하세요.

In [ ]:
# ===== Claude 4.5 모델 선택 =====
# 아래 세 가지 모델 중 하나를 선택하세요 (주석 해제)

# 1. Haiku 4.5 - 가장 빠르고 저렴 (테스트용 추천)
MODEL = "claude-haiku-4-5-20251001"

# 2. Sonnet 4.5 - 균형 잡힌 성능
# MODEL = "claude-sonnet-4-5-20250929"

# 3. Opus 4.5 - 최고 성능 (비용 높음)
# MODEL = "claude-opus-4-5-20251101"

print(f"선택된 모델: {MODEL}")

선택된 모델: claude-haiku-4-5-20251001


---

## 3. API 키 설정

In [ ]:
import os

# 방법 1: 직접 입력
# os.environ["ANTHROPIC_API_KEY"] = "sk-ant-여기에-API-키-입력"

# 방법 2: Colab Secrets
try:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
    print("Colab Secrets에서 API 키를 불러왔습니다!")
except:
    print("직접 API 키를 입력하세요.")

Colab Secrets에서 API 키를 불러왔습니다!


---

## 4. LangChain으로 Summary Memory 구현

### 4.1 기본 구성

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.chat_history import InMemoryChatMessageHistory

# LLM 초기화
llm = ChatAnthropic(
    model=MODEL,
    max_tokens=500
)

# 요약용 LLM (동일 모델 사용)
summarizer_llm = ChatAnthropic(
    model=MODEL,
    max_tokens=300
)

# 대화 요약 저장
conversation_summary = ""

# 최근 대화 (요약 전)
# 1/0 -> 2,0 ->3,0 -> 0,1 -> 1,1 -> 2,1 -> 3,1 -> 0,1
recent_messages = []

# 요약 트리거 (N개 메시지마다 요약)
SUMMARIZE_THRESHOLD = 4

print(f"Summary Memory 설정 완료! (요약 임계값: {SUMMARIZE_THRESHOLD}개 메시지)")

Summary Memory 설정 완료! (요약 임계값: 4개 메시지)


### 4.2 대화 요약 함수

In [ ]:
def summarize_conversation(messages: list, existing_summary: str = "") -> str:
    """
    대화 내용을 요약합니다.

    Parameters:
        messages: 요약할 메시지 리스트
        existing_summary: 기존 요약 (있으면 업데이트)

    Returns:
        str: 업데이트된 요약
    """
    # 메시지를 텍스트로 변환
    conversation_text = ""
    for msg in messages:
        role = "사용자" if isinstance(msg, HumanMessage) else "AI"
        conversation_text += f"{role}: {msg.content}\n"

    # 요약 프롬프트
    if existing_summary:
        prompt = f"""기존 대화 요약:
{existing_summary}

새로운 대화:
{conversation_text}

위 내용을 통합하여 간결하게 요약해주세요.
중요한 정보(이름, 선호도, 요청사항 등)를 포함하세요.
3-5문장으로 요약하세요."""
    else:
        prompt = f"""다음 대화를 간결하게 요약해주세요.
중요한 정보(이름, 선호도, 요청사항 등)를 포함하세요.
3-5문장으로 요약하세요.

대화:
{conversation_text}"""

    # llm.invoke는 언제나 []를 받는다.
    response = summarizer_llm.invoke([HumanMessage(content=prompt)])
    return response.content


# 테스트
test_messages = [
    HumanMessage(content="안녕! 내 이름은 민수야."),
    AIMessage(content="안녕하세요 민수님! 반갑습니다."),
    HumanMessage(content="좋아하는 음식은 피자야."),
    AIMessage(content="피자를 좋아하시는군요! 어떤 종류의 피자를 좋아하세요?"),
]

summary = summarize_conversation(test_messages)
print("대화 요약 테스트:")
print("-" * 50)
print(summary)

대화 요약 테스트:
--------------------------------------------------
# 대화 요약

민수는 자신을 소개했으며, 가장 좋아하는 음식은 피자입니다. AI는 민수를 환영하고 피자 선호도에 대해 더 자세히 알아보려고 했습니다.


### 4.3 Summary Memory 챗봇

In [ ]:
def chat_with_summary_memory(user_input: str) -> str:
    """
    Summary Memory 방식 챗봇.
    일정 메시지 수가 쌓이면 요약하여 저장합니다.
    """
    global conversation_summary, recent_messages

    # 사용자 메시지 추가
    recent_messages.append(HumanMessage(content=user_input))

    # 시스템 프롬프트 구성
    system_content = "당신은 친절한 AI 어시스턴트입니다. 한국어로 답변하세요."
    if conversation_summary:
        system_content += f"\n\n이전 대화 요약:\n{conversation_summary}"

    # 메시지 구성 (시스템 + 최근 대화)
    messages = [
        SystemMessage(content=system_content),
        *recent_messages
    ]

    # LLM 호출
    response = llm.invoke(messages)

    # AI 응답 저장
    recent_messages.append(AIMessage(content=response.content))

    # 임계값 초과 시 요약
    if len(recent_messages) >= SUMMARIZE_THRESHOLD:
        print(f"\n    [요약 트리거: {len(recent_messages)}개 메시지 → 요약 중...]")
        conversation_summary = summarize_conversation(recent_messages, conversation_summary)
        recent_messages = []  # 최근 메시지 초기화
        print(f"    [요약 완료]")

    return response.content


# 초기화
conversation_summary = ""
recent_messages = []

print("="*50)
print("Summary Memory 테스트")
print("="*50)

Summary Memory 테스트


### 4.4 테스트: 요약 후에도 정보 유지

In [ ]:
# 대화 시작
conversations = [
    "안녕! 내 이름은 지수야.",
    "좋아하는 색깔은 보라색이야.",  # 이 시점에서 요약 발생 (4개 메시지)
    "취미는 독서야.",
    "좋아하는 책 장르는 SF야.",  # 다시 요약 발생
    "영화는 미션임파서블을 좋아해",
    "음악은 힙합음악을 선호해",
    "음식은 파스타를 사랑해",
]

for i, msg in enumerate(conversations, 1):
    response = chat_with_summary_memory(msg)
    print(f"\n[{i}] 사용자: {msg}")
    print(f"    AI: {response[:100]}..." if len(response) > 100 else f"    AI: {response}")

print("\n" + "="*50)
print("요약된 정보 확인")
print("="*50)

# 요약된 정보 질문
response = chat_with_summary_memory("내 이름이랑 좋아하는 색깔이 뭐라고 했지?")
print(f"\n[5] 사용자: 내 이름이랑 좋아하는 색깔이 뭐라고 했지?")
print(f"    AI: {response}")


[1] 사용자: 안녕! 내 이름은 지수야.
    AI: 안녕, 지수! 반갑습니다! 😊

나는 Claude라고 해. 무엇을 도와드릴까?

    [요약 트리거: 4개 메시지 → 요약 중...]
    [요약 완료]

[2] 사용자: 좋아하는 색깔은 보라색이야.
    AI: 오, 보라색을 좋아하는군요! 🟣 

보라색은 정말 멋진 색이에요. 신비로우면서도 우아한 느낌이 있고, 창의성과 상상력을 나타내기도 하죠. 

지수님이 보라색을 좋아하는 이유가 뭐예...

[3] 사용자: 취미는 독서야.
    AI: 아, 그렇군요! 독서가 취미시군요. 😊

보라색을 좋아하고 독서를 즐기신다니, 정말 잘 어울리는 조합이에요. 보라색은 상상력과 창의성을 자극하는 색이라서 독서와 정말 잘 맞는 것 ...

    [요약 트리거: 4개 메시지 → 요약 중...]
    [요약 완료]

[4] 사용자: 좋아하는 책 장르는 SF야.
    AI: 오, SF군요! 정말 멋진데요! 🚀

SF는 정말 보라색과 잘 어울리는 장르 같아요. SF는 미래를 상상하고, 새로운 세계를 창조하고, 깊이 있는 철학적 질문을 던지는 장르니까요....

[5] 사용자: 영화는 미션임파서블을 좋아해
    AI: 아, 지수님이 미션임파서블을 좋아하시는군요! 흥미로운 선택이에요. 😊

미션임파서블은 SF와는 조금 다른 장르지만, 지수님의 취향을 더 잘 이해하는 데 도움이 됩니다:

**미션임...

    [요약 트리거: 4개 메시지 → 요약 중...]
    [요약 완료]

[6] 사용자: 음악은 힙합음악을 선호해
    AI: 오, 흥미로운 조합이네요! 🎵

지수님의 취향이 점점 더 명확해지고 있어요:
- **보라색** (신비롭고 창의적)
- **SF 독서** (상상력, 철학적 사고)
- **미션임파서블...

[7] 사용자: 음식은 파스타를 사랑해
    AI: 안녕하세요, 지수님! 🍝

파스타를 사랑하신다니 좋은 정보네요! 파스타의 어떤 매력이 끌리시나요?

제가 지수님을 더 잘 이해하기 위해 몇 가

### 4.5 현재 상태 확인

In [ ]:
print("현재 요약:")
print("-" * 50)
print(conversation_summary if conversation_summary else "(없음)")

print(f"\n최근 메시지 수: {len(recent_messages)}개")
if recent_messages:
    print("\n최근 메시지:")
    for i, msg in enumerate(recent_messages, 1):
        role = "사용자" if isinstance(msg, HumanMessage) else "AI"
        print(f"  [{i}] {role}: {msg.content[:50]}..." if len(msg.content) > 50 else f"  [{i}] {role}: {msg.content}")

현재 요약:
--------------------------------------------------
# 지수님 프로필 최종 요약

**기본 정보:**
- 이름: 지수
- 선호색: 보라색 (신비롭고 우아하며 창의성을 나타냄)

**취향:**
지수님은 창의성과 자유로운 표현을 중시하는 사람으로, SF 독서, 미션임파서블 영화, 힙합음악, 그리고 **파스타**를 사랑합니다. 강한 메시지와 스토리텔링을 선호하며, 현재 구체적인 SF 작가/작품, SF 세부 장르, 선호하는 힙합 아티스트, 그리고 파스타 선호 유형(소스 종류 및 좋아하는 이유)에 대한 추가 정보 수집이 필요합니다.

최근 메시지 수: 0개


---

## 5. LangGraph로 Summary Memory 구현

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

# 상태 정의 (요약 포함)
class SummaryState(TypedDict):
    messages: Annotated[list, add_messages]  # 최근 메시지
    summary: str  # 대화 요약


def should_summarize(state: SummaryState) -> str:
    """요약이 필요한지 결정"""
    if len(state["messages"]) >= 6:  # 6개 메시지 초과 시
        return "summarize"
    return "respond"


def summarize_node(state: SummaryState) -> dict:
    """대화 요약 노드"""
    # 메시지를 텍스트로 변환
    conversation_text = ""
    for msg in state["messages"]:
        role = "사용자" if isinstance(msg, HumanMessage) else "AI"
        conversation_text += f"{role}: {msg.content}\n"

    existing_summary = state.get("summary", "")

    if existing_summary:
        prompt = f"""기존 요약: {existing_summary}\n\n새 대화: {conversation_text}\n\n통합 요약 (3-5문장):"""
    else:
        prompt = f"""대화 요약 (3-5문장):\n{conversation_text}"""

    response = summarizer_llm.invoke([HumanMessage(content=prompt)])

    # 최근 2개 메시지만 유지하고 나머지는 요약으로 대체
    return {
        "summary": response.content,
        "messages": state["messages"][-2:]  # 마지막 2개만 유지
    }


def chatbot_node(state: SummaryState) -> dict:
    """LLM 응답 노드"""
    system_content = "당신은 친절한 AI 어시스턴트입니다. 한국어로 답변하세요."

    summary = state.get("summary", "")
    if summary:
        system_content += f"\n\n이전 대화 요약:\n{summary}"

    messages = [
        SystemMessage(content=system_content),
        *state["messages"]
    ]

    response = llm.invoke(messages)
    return {"messages": [response]}


# 그래프 구성
graph_builder = StateGraph(SummaryState)
graph_builder.add_node("summarize", summarize_node)
graph_builder.add_node("chatbot", chatbot_node)

# 조건부 라우팅
graph_builder.add_conditional_edges(
    START,
    should_summarize,
    {"summarize": "summarize", "respond": "chatbot"}
)
graph_builder.add_edge("summarize", "chatbot")
graph_builder.add_edge("chatbot", END)

# 컴파일
memory_saver = MemorySaver()
graph = graph_builder.compile(checkpointer=memory_saver)

print("LangGraph Summary Memory 설정 완료!")

LangGraph Summary Memory 설정 완료!


### 5.1 LangGraph Summary Memory 테스트

In [ ]:
config = {"configurable": {"thread_id": "summary-session-1"}}

def chat_with_langgraph_summary(user_input: str) -> str:
    """LangGraph 기반 Summary Memory 챗봇"""
    result = graph.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=config
    )
    return result["messages"][-1].content


print("="*50)
print("LangGraph Summary Memory 테스트")
print("="*50)

# 여러 대화 진행
conversations = [
    "안녕! 내 이름은 현우야.",
    "나는 대구에 살아.",
    "좋아하는 음식은 삼겹살이야.",
    "취미는 등산이야.",
    "좋아하는 영화는 인터스텔라야.",
]

for i, msg in enumerate(conversations, 1):
    response = chat_with_langgraph_summary(msg)
    print(f"\n[{i}] 사용자: {msg}")
    print(f"    AI: {response[:80]}..." if len(response) > 80 else f"    AI: {response}")

print("\n" + "="*50)
print("요약된 정보 확인")
print("="*50)

response = chat_with_langgraph_summary("내가 어디 살고 뭘 좋아한다고 했지?")
print(f"\n[6] 사용자: 내가 어디 살고 뭘 좋아한다고 했지?")
print(f"    AI: {response}")

LangGraph Summary Memory 테스트

[1] 사용자: 안녕! 내 이름은 현우야.
    AI: 안녕하세요, 현우님! 😊

만나서 반갑습니다! 저는 여러분을 도와드릴 수 있는 AI 어시스턴트입니다. 

무엇을 도와드릴까요?

[2] 사용자: 나는 대구에 살아.
    AI: 그렇군요! 현우님은 대구에 사시는군요. 😊

대구는 한반도의 중심에 있는 아름다운 도시네요. 혹시 대구에 대해 궁금한 점이 있으신가요? 아니면 ...

[3] 사용자: 좋아하는 음식은 삼겹살이야.
    AI: 오, 삼겹살이요! 정말 맛있는 음식이네요! 🍖

삼겹살은 고소한 맛과 쫄깃한 식감이 정말 좋죠. 특히 숯불에 구워 먹을 때 그 맛이 최고라고 많...

[4] 사용자: 취미는 등산이야.
    AI: 와, 등산이 취미시군요! 정말 좋은 취미예요! ⛰️

등산은 자연을 느끼면서 건강도 챙길 수 있는 멋진 활동이네요.

현우님은 등산을 자주 하시...

[5] 사용자: 좋아하는 영화는 인터스텔라야.
    AI: 오, 인터스텔라요! 정말 훌륭한 영화 선택이네요! 🚀🌌

인터스텔라는 정말 감동적이고 웅장한 SF 영화죠. 우주 탐험, 사랑, 인류의 미래라는 ...

요약된 정보 확인

[6] 사용자: 내가 어디 살고 뭘 좋아한다고 했지?
    AI: 좋은 질문이네요! 😊

현우님이 말씀해 주신 것들을 정리하면:

**📍 거주지:** 대구

**🍖 좋아하는 음식:** 삼겹살

**⛰️ 취미:** 등산

**🎬 좋아하는 영화:** 인터스텔라

이 네 가지를 말씀해 주셨어요! 혹시 제가 놓친 게 있거나, 추가로 더 알려주고 싶으신 게 있으신가요?


### 5.2 저장된 상태 확인

In [ ]:
state = graph.get_state(config)

print("현재 요약:")
print("-" * 50)
print(state.values.get("summary", "(없음)"))

print(f"\n현재 메시지 수: {len(state.values['messages'])}개")
print("\n최근 메시지:")
for i, msg in enumerate(state.values['messages'], 1):
    role = "사용자" if isinstance(msg, HumanMessage) else "AI"
    content = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
    print(f"  [{i}] {role}: {content}")

현재 요약:
--------------------------------------------------
# 통합 요약

**현우님의 개인정보 및 선호도:**

현우님은 대구에 거주하며, 좋아하는 음식은 삼겹살입니다. 취미는 등산이고, 좋아하는 영화는 인터스텔라로 SF 영화를 즐기시는 것으로 보입니다. 현우님은 자연을 사랑하면서도 우주와 같은 거대한 스케일의 이야기에 관심이 있는 것으로 파악됩니다.

현재 메시지 수: 12개

최근 메시지:
  [1] 사용자: 안녕! 내 이름은 현우야.
  [2] AI: 안녕하세요, 현우님! 😊

만나서 반갑습니다! 저는 여러분을 도와드릴 수 있는 AI 어시스...
  [3] 사용자: 나는 대구에 살아.
  [4] AI: 그렇군요! 현우님은 대구에 사시는군요. 😊

대구는 한반도의 중심에 있는 아름다운 도시네요...
  [5] 사용자: 좋아하는 음식은 삼겹살이야.
  [6] AI: 오, 삼겹살이요! 정말 맛있는 음식이네요! 🍖

삼겹살은 고소한 맛과 쫄깃한 식감이 정말 ...
  [7] 사용자: 취미는 등산이야.
  [8] AI: 와, 등산이 취미시군요! 정말 좋은 취미예요! ⛰️

등산은 자연을 느끼면서 건강도 챙길 ...
  [9] 사용자: 좋아하는 영화는 인터스텔라야.
  [10] AI: 오, 인터스텔라요! 정말 훌륭한 영화 선택이네요! 🚀🌌

인터스텔라는 정말 감동적이고 웅장...
  [11] 사용자: 내가 어디 살고 뭘 좋아한다고 했지?
  [12] AI: 좋은 질문이네요! 😊

현우님이 말씀해 주신 것들을 정리하면:

**📍 거주지:** 대구
...


---

## 6. Summary Memory 특징

### 장점
- 적은 토큰으로 전체 맥락 유지
- 긴 대화에서도 효율적
- 중요 정보 보존

### 단점
- 요약 과정에서 세부 정보 손실 가능
- 요약을 위한 추가 LLM 호출 필요
- 요약 품질에 의존

### 요약 트리거 전략

| 전략 | 설명 |
|------|------|
| 메시지 수 | N개 메시지마다 요약 |
| 토큰 수 | N토큰 초과 시 요약 |
| 주기적 | N분마다 요약 |

### 다음 단계

**04_window_summary_memory.ipynb**에서는 Window와 Summary를 결합한 하이브리드 방식을 배웁니다.